# Demostración Integral: Detección de Regiones Brillantes (Lámparas)

## Objetivo
Implementar y comparar múltiples algoritmos de visión computacional para la detección automática de lámparas en imágenes, demostrando la efectividad de diferentes enfoques técnicos.

## Algoritmos Implementados
1. **Pipeline Principal**: Detección integral con umbral adaptativo
2. **Comparación de Umbrales**: Fijo vs Adaptativo vs Otsu
3. **Filtros de Pre-procesamiento**: Gaussian Blur y reducción de ruido
4. **Operaciones Morfológicas**: Apertura, cierre y refinamiento
5. **Segmentación Avanzada**: K-means y crecimiento de regiones
6. **Análisis Cuantitativo**: Métricas de área, perímetro y centroide

In [8]:
# Importar librerías necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from sklearn.cluster import KMeans
import os

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

# Lista de imágenes a procesar
imagenes = ['../image/1.png', '../image/2.png', '../image/3.png', '../image/4.png']

# Crear directorio de resultados
os.makedirs('../image/resultados/demostracion/', exist_ok=True)

print("✅ Librerías cargadas correctamente")
print(f"📁 Procesaremos {len(imagenes)} imágenes")

✅ Librerías cargadas correctamente
📁 Procesaremos 4 imágenes


## 1. ALGORITMO PRINCIPAL: Pipeline Integral de Detección

Este es nuestro algoritmo estrella que combina múltiples técnicas optimizadas específicamente para la detección de lámparas.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import label, filters, morphology, measure
from skimage.feature import peak_local_maxima
from skimage.segmentation import watershed
from scipy import ndimage
from sklearn.cluster import DBSCAN

def detectar_lamparas_mejorado(imagen_path, mostrar_pasos=True, debug=False):
    """
    Pipeline mejorado para detección de zonas iluminadas con mayor precisión
    """
    # 1. Cargar y preprocesar imagen
    img_color = cv2.imread(imagen_path)
    if img_color is None:
        return None, None
    
    # Convertir a diferentes espacios de color para análisis
    gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(img_color, cv2.COLOR_BGR2LAB)
    
    # 2. Detección de regiones muy brillantes (candidatas a lámparas)
    # Usar canal L de LAB que representa mejor la luminancia
    luminance = lab[:,:,0]
    
    # Umbral dinámico basado en percentiles
    umbral_alto = np.percentile(luminance, 95)  # Top 5% más brillante
    umbral_medio = np.percentile(luminance, 85) # Top 15% brillante
    
    # Máscara de regiones muy brillantes (núcleos de lámparas)
    mascara_nucleos = luminance > umbral_alto
    
    # Máscara de regiones iluminadas (halo de luz)
    mascara_halo = luminance > umbral_medio
    
    # 3. Filtrado por saturación (las lámparas tienden a tener baja saturación)
    saturacion = hsv[:,:,1]
    mascara_baja_saturacion = saturacion < 80  # Ajustable según el tipo de luz
    
    # Combinar criterios
    candidatos_lamparas = mascara_nucleos & mascara_baja_saturacion
    
    # 4. Operaciones morfológicas mejoradas
    # Kernel circular para mejor preservación de formas redondeadas
    kernel_circular = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    kernel_grande = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    
    # Limpiar ruido manteniendo las formas principales
    candidatos_limpios = cv2.morphologyEx(candidatos_lamparas.astype(np.uint8), 
                                         cv2.MORPH_OPEN, kernel_circular)
    candidatos_limpios = cv2.morphologyEx(candidatos_limpios, 
                                         cv2.MORPH_CLOSE, kernel_grande)
    
    # 5. Detección de zonas de influencia usando Watershed
    # Calcular distancia desde bordes
    dist_transform = cv2.distanceTransform(candidatos_limpios, cv2.DIST_L2, 5)
    
    # Encontrar picos locales (centros de lámparas)
    picos = peak_local_maxima(dist_transform, min_distance=20, threshold_abs=0.3)
    
    # Crear marcadores para watershed
    marcadores = np.zeros(dist_transform.shape, dtype=np.int32)
    for i, (y, x) in enumerate(picos[0]):
        marcadores[y, x] = i + 1
    
    # Aplicar watershed para segmentar zonas de influencia
    if len(picos[0]) > 0:
        # Invertir la imagen para watershed (valles = zonas brillantes)
        imagen_watershed = 255 - luminance
        labels_watershed = watershed(imagen_watershed, marcadores, mask=mascara_halo)
    else:
        labels_watershed = candidatos_limpios
    
    # 6. Análisis de regiones mejorado con múltiples criterios
    props = measure.regionprops(labels_watershed, intensity_image=luminance)
    
    lamparas_detectadas = []
    result = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)
    
    for region in props:
        # Filtros mejorados para validar lámparas
        area = region.area
        intensidad_media = region.mean_intensity
        
        # Criterios de validación
        area_valida = 50 < area < 5000  # Rango de área razonable
        intensidad_alta = intensidad_media > np.percentile(luminance, 70)
        
        # Factor de forma (circularidad)
        perimeter = region.perimeter
        if perimeter > 0:
            circularidad = 4 * np.pi * area / (perimeter ** 2)
            forma_razonable = circularidad > 0.3  # Más permisivo que un círculo perfecto
        else:
            forma_razonable = False
        
        # Relación aspecto (evitar líneas muy alargadas)
        minr, minc, maxr, maxc = region.bbox
        aspecto = (maxc - minc) / max((maxr - minr), 1)
        aspecto_razonable = 0.3 < aspecto < 3.0
        
        if area_valida and intensidad_alta and forma_razonable and aspecto_razonable:
            y0, x0 = region.centroid
            
            # Calcular zona de influencia de la luz
            zona_influencia = calcular_zona_influencia(luminance, (int(x0), int(y0)), 
                                                     umbral_medio)
            
            # Dibujar resultado mejorado
            cv2.rectangle(result, (minc, minr), (maxc, maxr), (255, 0, 0), 2)
            cv2.circle(result, (int(x0), int(y0)), 3, (0, 255, 0), -1)
            
            # Mostrar zona de influencia
            if zona_influencia is not None:
                contornos, _ = cv2.findContours(zona_influencia.astype(np.uint8), 
                                              cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(result, contornos, -1, (255, 255, 0), 1)
            
            cv2.putText(result, f'L{len(lamparas_detectadas)+1}', 
                       (minc, minr-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            lamparas_detectadas.append({
                'id': len(lamparas_detectadas) + 1,
                'centroide': (x0, y0),
                'area': area,
                'intensidad_media': intensidad_media,
                'circularidad': circularidad,
                'zona_influencia_area': np.sum(zona_influencia) if zona_influencia is not None else 0
            })
    
    # 7. Post-procesamiento: eliminar duplicados cercanos
    lamparas_filtradas = eliminar_duplicados(lamparas_detectadas, distancia_min=30)
    
    if mostrar_pasos:
        mostrar_proceso_mejorado(img_color, gray, luminance, mascara_nucleos, 
                               mascara_halo, candidatos_limpios, dist_transform, 
                               result, lamparas_filtradas)
    
    return result, lamparas_filtradas

def calcular_zona_influencia(luminance, centro, umbral_minimo):
    """
    Calcula la zona de influencia de una lámpara usando crecimiento de regiones
    """
    try:
        x, y = centro
        h, w = luminance.shape
        
        # Crear máscara inicial en el punto central
        mask = np.zeros_like(luminance, dtype=bool)
        if 0 <= y < h and 0 <= x < w:
            mask[y, x] = True
        
        # Crecimiento de región iterativo
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        
        for _ in range(20):  # Máximo 20 iteraciones
            mask_expandida = cv2.dilate(mask.astype(np.uint8), kernel, iterations=1)
            
            # Solo incluir píxeles que superen el umbral
            nueva_mask = (mask_expandida > 0) & (luminance > umbral_minimo)
            
            if np.array_equal(nueva_mask, mask):
                break  # Convergencia alcanzada
            
            mask = nueva_mask
        
        return mask
    except:
        return None

def eliminar_duplicados(lamparas, distancia_min=30):
    """
    Elimina lámparas duplicadas que están muy cerca entre sí
    """
    if len(lamparas) <= 1:
        return lamparas
    
    if DBSCAN is not None:
        # Usar DBSCAN si está disponible
        coordenadas = np.array([lamp['centroide'] for lamp in lamparas])
        clustering = DBSCAN(eps=distancia_min, min_samples=1).fit(coordenadas)
        labels = clustering.labels_
        
        # Mantener solo el mejor representante de cada grupo
        lamparas_filtradas = []
        for cluster_id in set(labels):
            indices_cluster = np.where(labels == cluster_id)[0]
            mejor_lampara = max([lamparas[i] for i in indices_cluster], 
                               key=lambda x: x['intensidad_media'])
            lamparas_filtradas.append(mejor_lampara)
        
        return lamparas_filtradas
    
    else:
        # Método alternativo sin sklearn
        lamparas_filtradas = []
        lamparas_restantes = lamparas.copy()
        
        while lamparas_restantes:
            lampara_actual = lamparas_restantes.pop(0)
            lamparas_filtradas.append(lampara_actual)
            
            # Eliminar lámparas cercanas
            x1, y1 = lampara_actual['centroide']
            lamparas_restantes = [
                lamp for lamp in lamparas_restantes
                if np.sqrt((lamp['centroide'][0] - x1)**2 + (lamp['centroide'][1] - y1)**2) > distancia_min
            ]
        
        return lamparas_filtradas

def mostrar_proceso_mejorado(img_color, gray, luminance, mascara_nucleos, 
                           mascara_halo, candidatos_limpios, dist_transform, 
                           result, lamparas):
    """
    Visualización del proceso mejorado paso a paso
    """
    fig, axes = plt.subplots(3, 3, figsize=(20, 15))
    axes = axes.flatten()
    
    # Imagen original
    axes[0].imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
    axes[0].set_title('1. Imagen Original')
    axes[0].axis('off')
    
    # Canal de luminancia
    axes[1].imshow(luminance, cmap='gray')
    axes[1].set_title('2. Canal L (Luminancia)')
    axes[1].axis('off')
    
    # Núcleos de lámparas
    axes[2].imshow(mascara_nucleos, cmap='Reds')
    axes[2].set_title('3. Núcleos Brillantes (Top 5%)')
    axes[2].axis('off')
    
    # Halo de luz
    axes[3].imshow(mascara_halo, cmap='Oranges')
    axes[3].set_title('4. Zonas Iluminadas (Top 15%)')
    axes[3].axis('off')
    
    # Candidatos después de morfología
    axes[4].imshow(candidatos_limpios, cmap='Greens')
    axes[4].set_title('5. Candidatos Filtrados')
    axes[4].axis('off')
    
    # Transform de distancia
    axes[5].imshow(dist_transform, cmap='hot')
    axes[5].set_title('6. Transform de Distancia')
    axes[5].axis('off')
    
    # Resultado final
    axes[6].imshow(result)
    axes[6].set_title(f'7. Resultado Final ({len(lamparas)} lámparas)')
    axes[6].axis('off')
    
    # Estadísticas
    axes[7].axis('off')
    stats_text = "Estadísticas:\n\n"
    for i, lamp in enumerate(lamparas[:5]):
        stats_text += f"L{lamp['id']}:\n"
        stats_text += f"  Área: {lamp['area']:.0f} px\n"
        stats_text += f"  Intensidad: {lamp['intensidad_media']:.1f}\n"
        stats_text += f"  Circularidad: {lamp['circularidad']:.2f}\n"
        stats_text += f"  Zona influencia: {lamp['zona_influencia_area']:.0f} px\n\n"
    
    if len(lamparas) > 5:
        stats_text += f"... y {len(lamparas)-5} lámparas más"
    
    axes[7].text(0.1, 0.9, stats_text, transform=axes[7].transAxes, 
                fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    # Ocultar el último subplot si no se usa
    axes[8].axis('off')
    
    plt.tight_layout()
    plt.show()

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar una imagen
    imagen_path = "path/to/your/image.jpg"
    resultado, lamparas = detectar_lamparas_mejorado(imagen_path, mostrar_pasos=True)
    
    if resultado is not None:
        print(f"✅ Detectadas {len(lamparas)} lámparas con el algoritmo mejorado")
        
        # Guardar resultado
        # cv2.imwrite('resultado_mejorado.png', cv2.cvtColor(resultado, cv2.COLOR_RGB2BGR))
    else:
        print("❌ Error al procesar la imagen")

ImportError: cannot import name 'label' from 'skimage' (/home/yep/Proyectos/parcial1_visualizacion/.venv/lib64/python3.13/site-packages/skimage/__init__.py)